# Binding Energy
You are going to analyse the binding energies of the known isotopes.

#### Reading data
The data is read from a *parquet* file. This file format contains the datatypes (e.g. int or float) in addition to the actual values.

In [ ]:
import polars as pl

isotopes = pl.read_parquet('data/binding_energy.parquet')

#### Browsing the data
Have a first look at the dataframe and find out about the different columns, the number of isotopes, etc.

In [ ]:
display(isotopes.head(10)) # display 10 random rows

In [ ]:
rows, cols = isotopes.shape
print(f'{rows} rows and {cols} columns')

#### Some search tasks
Answer the following questions:
- Which isotope has the greates atomic mass?
- How many carbon isotopes are known?
- Which element has the greatest number of isotopes?
- Which lead (Pb) isotopes have a non-negligible abundance (column *Isotopic Composition*) and do their abundances add up to 100 %?

In [ ]:
# find the isotope with the maximum atomic mass (Og stands for Oganessum)
isotopes.filter(pl.col('Atomic Mass') == pl.max('Atomic Mass'))

In [ ]:
# filter carbon (C) isotopes; there are 16 rows
isotopes.filter(pl.col('Atomic Symbol') == 'C')

In [ ]:
# group by atomic number, aggregate number of isotopes (len), sort by number of isotopes (descending)
(isotopes
    .group_by('Z')
    .agg([
        pl.col('Atomic Symbol'), 
        pl.len().alias('Number of Isotopes')]
        )
    .with_columns(pl.col('Atomic Symbol').list.first())
    .sort('Number of Isotopes', descending=True)
) 

### Mass Excess
The *mass excess* (or *mass defect*) corresponds to the difference between the measured mass ('Atomic Mass', unit µu) and the mass number ('Mass Number', unit u). In the dataframe it is expressed as the equivalent energy in keV. Add another column to verify this relation.

In [ ]:
from scipy.constants import eV, m_u, speed_of_light

(isotopes
    .with_columns(
        ((pl.col('Atomic Mass') * 1e-6 - pl.col('Mass Number')) * m_u * speed_of_light**2 /(1000 * eV))
            .alias('My Mass Excess'))
    .with_columns((pl.col('Mass Excess') / pl.col('My Mass Excess')).alias('rel'))
    .select(['Atomic Symbol', 'N', 'Mass Excess', 'My Mass Excess', 'rel'])
).sort('rel')

### Nuclear Binding Energy
The nuclear binding energy of a nucleus can be approximated as the mass difference of its measured mass ('Atomic Mass') and the sum of the masses of the protons, neutrons and electrons, multiplied by the square of the speed of light.

Add columns for the binding energy (in MeV) and the binding energy divided by the mass number. Compare the calculated value to the values in the column 'Binding Energy per Nucleus' in the dataframe. The deviations are mostly due the electronic binding energy, which has been neglected in this calculation.

In [ ]:
from scipy.constants import m_n, m_p, m_e, u, speed_of_light, eV

def e_bind(Z, N, m):
    """
    function to calculate the binding energy of a nucleus
    Z: atomic number (number of protons)
    N: neutron number
    m: atomic mass (in µu)
    """
    
    dm = Z * (m_p + m_e) + N * m_n - m * 1e-6 * u # mass difference in kg
    eb = dm * speed_of_light**2 / eV # binding energy in eV
    
    return eb / 1e6 # return binding energy in MeV


# add columns for binding energy and binding energy per nucleon
binding_energy = isotopes.with_columns(
    pl.struct(['Z', 'N', 'Atomic Mass'])
    .map_elements(lambda s: e_bind(s['Z'], s['N'], s['Atomic Mass']), return_dtype=pl.Float64)
    .alias('Binding Energy')
).with_columns(
    (pl.col('Binding Energy') / pl.col('Mass Number'))
        .alias('My Binding Energy per Nucleon'),
    pl.col('Binding Energy per Nucleon') / 1000
)

# dataframe with only highest binding energy for each mass number (used for upper envelope)
binding_energy_max = (binding_energy
                      .sort('Binding Energy per Nucleon')
                      .group_by('Mass Number')
                      .agg([
                          pl.col('Z').last(),
                          pl.col('N').last(),
                          pl.col('Atomic Symbol').last(), 
                          pl.col('Binding Energy per Nucleon').max()
                      ])
                     )

In [ ]:
(binding_energy.select(['Atomic Symbol', 'Z', 'Mass Number',
                       pl.col('Binding Energy per Nucleon'), 'My Binding Energy per Nucleon', 
                       (pl.col('Binding Energy per Nucleon') / pl.col('My Binding Energy per Nucleon'))
                           .abs().alias('rel')])
    .sort('rel', descending=True)
)

#### Graph
Graph the nuclear binding energy per nucleon in a diagram. Find the isotope with the highest binding energy per nucleon, i.e. the most stable isotope. List the isotopes with a negative binding energy.

In [ ]:
import altair as alt

# chart1 with all data points
chart1 = (binding_energy.plot.point(
            x=alt.X('Mass Number',
                    scale=alt.Scale(type='log', domain=(1, 300)) # logarithmic scale (looks better)
                   ),
            y='Binding Energy per Nucleon',
            tooltip=[
                'Atomic Symbol', 
                'Mass Number', 
                alt.Tooltip('Binding Energy per Nucleon', format='.2f') # round values to 2 decimal places in tooltips
            ]
            )
          .properties(width=600, title='Binding Energy per Nucleon')
        )


#chart2 with only max for each mass number
chart2 = (binding_energy_max.plot.line(
            x=alt.X('Mass Number',
                    scale=alt.Scale(type='log', domain=(1, 300))
                   ),
            y=alt.Y('Binding Energy per Nucleon',
                   title='Binding Energy per Nucleon (MeV)'
                   ),
            tooltip=['Atomic Symbol', 'Mass Number', 'Binding Energy per Nucleon']
            )
        )

chart = ((chart1 + chart2)
           .configure_point(size=1).configure_line(color='red') # format points and line
       )

chart

In [ ]:
binding_energy.filter(pl.col('Binding Energy per Nucleon') == pl.max('Binding Energy per Nucleon'))

The most stable isotope (isotope with highest binding energy per nucleon) is Ni-62.

In [ ]:
binding_energy.filter(pl.col('Binding Energy per Nucleon') < 0)

In [ ]:
def droplet(N, Z):
    
    A = N + Z # number of nucleons

    pairing = 'e' if N % 2 == 0 else 'o'
    pairing += 'e' if Z % 2 == 0 else 'o'
    match pairing:
        case 'ee':
            aP = 11.18
        case 'oo':
            aP = -11.18
        case _:
            aP = 0
            

    EV = 15.76 * A
    EO = -17.81 * A**(2/3)
    EC = - 0.711 * Z * (Z - 1) * A**(-1/3)
    ES = -23.702* (N - Z)**2 / (4 * A)
    EP = aP * A**(-1/2)

    return (EV + EO + EC + ES + EP) / A

In [ ]:
droplet = binding_energy_max.with_columns(pl.struct(['Z', 'N']).map_elements(lambda s: droplet(s['Z'], s['N']), return_dtype=pl.Float64).alias('Droplet'))

In [ ]:
#chart3 with binding energy predicted by droplet model (only envelope)
chart3 = (droplet.plot.line(
            x=alt.X('Mass Number',
                    scale=alt.Scale(type='log', domain=(1, 300))
                   ),
            y=alt.Y('Droplet',
                   title='Binding Energy per Nucleon (MeV)'
                   ),
            tooltip=['Atomic Symbol', 'Mass Number', 'Droplet']
            )
        )

In [ ]:
chart4 = chart + chart3

In [ ]:
chart4